In [1]:
# 08 June 2021, Tuesday
# Deploying ResNet model trained using APTOS cropped and preprocessed images

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import os

import PIL
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform

In [2]:
import azureml
from azureml.core import Workspace

print("Azure ML SDK Version: ", azureml.core.VERSION)

ws = Workspace(subscription_id="a02597e4-584a-47b3-99d6-529cb1a17c03",
               workspace_name='az-workspace',
               resource_group="azure-for-students")

Azure ML SDK Version:  1.28.0


In [3]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'a02597e4-584a-47b3-99d6-529cb1a17c03'
resource_group = 'azure-for-students'
workspace_name = 'az-workspace'

# workspace = Workspace(subscription_id, resource_group, workspace_name)

# dataset = Dataset.get_by_name(ws, name='dr-color-train')
# dataset.download(target_path='./dr-color-train/', overwrite=False)

In [4]:
"""
# Creating GPU Cluster (For training)
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'gpu-cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())
"""

"\n# Creating GPU Cluster (For training)\nfrom azureml.core.compute import ComputeTarget, AmlCompute\nfrom azureml.core.compute_target import ComputeTargetException\n\ncluster_name = 'gpu-cluster'\n\ntry:\n    compute_target = ComputeTarget(workspace=ws, name=cluster_name)\n    print('Found existing compute target')\nexcept ComputeTargetException:\n    print('Creating a new compute target...')\n    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', \n                                                           max_nodes=4)\n    \n    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)\n\ncompute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)\n\nprint(compute_target.get_status().serialize())\n"

In [4]:
# RESNET MODEL

def res_block(X, filter, stage):
    # Convolutional block
    X_copy = X
    f1, f2, f3 = filter

    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = MaxPool2D((2,2))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)

    # Short Path
    X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
    X_copy = MaxPool2D((2,2))(X_copy)
    X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    # Identity Block 1
    X_copy = X


    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    # Identity Block 2
    X_copy = X


    # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

    # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    return X

In [5]:
input_shape = (512, 512, 3)

X_input = Input(input_shape)

X = ZeroPadding2D((3,3))(X_input)

# 1 - stage

X = Conv2D(64, (7,7), strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides= (2,2))(X)

# 2- stage

X = res_block(X, filter= [64,64,256], stage= 2)

# 3- stage

X = res_block(X, filter= [128,128,512], stage= 3)

# 4- stage

X = res_block(X, filter= [256,256,1024], stage= 4)

# # 5- stage

X = res_block(X, filter= [512,512,2048], stage= 5)

#Average Pooling

X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

#Final layer

X = Flatten()(X)
X = Dense(2, activation = 'softmax', name = 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)


model = Model(inputs= X_input, outputs = X, name = 'Resnet18')

model.summary()

Model: "Resnet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [6]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics= ['accuracy'])

In [63]:
# earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
# checkpointer = ModelCheckpoint(filepath="/kaggle/working/resnet-azure-1-weights.hdf5", verbose=1, save_best_only=True)

In [12]:
pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 100.1 MB/s eta 0:00:01
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.2 which is incompatible.
ERROR: tensorflow-gpu 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.2 which is incompatible.
ERROR: raiwidgets 0.2.2 has requirement lightgbm>=3.1.1, but you'll have lightgbm 2.3.0 which is incompatible.
ERROR: gym 0.18.0 has requirement Pillow<=7.2.0, but you'll have pillow 8.2.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.28.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: azureml-opendatasets 1.28.0 has requirement numpy<=1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: azureml-opendatasets 1.28.0 has requirement scipy<=1.4.1,>=1.0.0, but you'll have scipy 1.5.2 which is incompatible.
ERR

In [7]:
model.load_weights('./preprocess-binary-aptos-resnet-5-100-weights.hdf5')

In [8]:
model.save('dr-resnet-bp-aptos.pkl')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: dr-resnet-bp-aptos.pkl/assets


In [9]:
import os
os.environ['TF_KERAS'] = '1'
import onnxmltools

onnx_model = onnxmltools.convert_keras(model) 

onnxmltools.utils.save_model(onnx_model, 'aptos_binary_prepr_resnet_model.onnx')


Can't import tf2onnx module, so the conversion on a model with any custom/lambda layer will fail!


In [10]:

from azureml.core.model import Model

model = Model.register(model_path = "aptos_binary_prepr_resnet_model.onnx",
                       model_name = "MyDRModelAPTOS",
                       description = "Keras Model for Diabetic Retinopathy Detection, APTOS cropped and preprocessed dataset (8 June)",
                       workspace = ws)

Registering model MyDRModelAPTOS


In [11]:
%%writefile score.py

import json
import sys

from azureml.core.model import Model
import onnxruntime
import numpy as np

def init():
    global model_path
    model_path = Model.get_model_path(model_name = 'MyDRModelAPTOS')

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data, dtype=np.float32)

        session = onnxruntime.InferenceSession(model_path)
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        result = session.run([output_name], {input_name: data})
        # NumPy arrays are not JSON serialisable
        result = result[0].tolist()

        return {"result": result}
    except Exception as e:
        result = str(e)
        return {"error": result}


Overwriting score.py


In [12]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("onnxruntime")
myenv.add_conda_package("pip==20.1.1")
# myenv.add_pip_package("ruamel")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [13]:

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "test"
                                                 )

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  


In [14]:
image = ContainerImage.create(name = "onnxmodelimage-aptos",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  after removing the cwd from sys.path.


Creating image
Running........................................
Succeeded
Image creation operation finished for image onnxmodelimage-aptos:1, operation "Succeeded"


In [15]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "diabetic-retinopathy-aptos", "type": "multilclass-classification"}, 
                                               description = 'Diabetic Retinopathy Detection ResNet binary classification model trained with APTOS images cropped and preprocessed',
                                               auth_enabled=True)

service_name = 'diabetic-retinopathy-aptos'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)

service.wait_for_deployment(show_output = True)
print(service.state)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  del sys.path[0]


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-06-08 05:22:51+00:00 Generating deployment configuration.
2021-06-08 05:22:52+00:00 Submitting deployment to compute.
2021-06-08 05:22:55+00:00 Checking the status of deployment diabetic-retinopathy-aptos..
2021-06-08 05:23:44+00:00 Checking the status of inference endpoint diabetic-retinopathy-aptos.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [16]:
print(service.get_logs())

2021-06-08T05:23:30,705242400+00:00 - rsyslog/run 
2021-06-08T05:23:30,706689300+00:00 - gunicorn/run 
2021-06-08T05:23:30,713987400+00:00 - iot-server/run 
2021-06-08T05:23:30,727295900+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-06-08T05:23:31,315942000+00:00 - iot-server/finish 1 0
2021-06-08T05:23:31,317264500+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (60)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 94
Initializing logger
2021-06-08 05:23:37,894 | root | INFO | Starting up app insights client
Starting up app insights client
2021-06-08 05:23:37,895 | root | INFO | Starting up request id generator
Starting up request id generator
2021-06-08 05:23:37,895 | root | INFO | Starting up app insight hooks
Starting up app insight hooks
2021-06-08 05:23:37,896 | root | INFO | Invoking user's init function
Invoking user's init functio

In [3]:
# print(ws.webservices['diabetic-retinopathy-idrid'].get_logs())

service = ws.webservices['diabetic-retinopathy-aptos']

In [4]:
service.update(enable_app_insights=True)

In [5]:
service.update(enable_app_insights=False)

In [6]:
service.update(auth_enabled=False)

In [24]:
import requests
import json
import numpy as np
from PIL import Image
from numpy import asarray


In [28]:
headers = {'Content-Type':'application/json'}

# path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/diabetic-retinopathy/code/Users/joannejons/test-deploy-idrid/'

test_img = [('IDRiD_029.jpeg', '0'), ('IDRiD_030.jpeg', '0'), ('IDRiD_101.jpeg', '1'), ('IDRiD_085.jpeg', '1'), ('IDRiD_008.jpeg', '2'), ('IDRiD_009.jpeg', '2'), ('IDRiD_006.jpeg', '3'), ('IDRiD_006.jpeg', '3'), ('IDRiD_001.jpeg', '4'), ('IDRiD_002.jpeg', '4')]

path = './test-deploy-idrid/'


for img_name, label in test_img:
    img = Image.open(path + img_name)
    img = asarray(img, dtype='float32')
    img = img / 255
    img = img.reshape(-1, 512, 512, 3)

    input_data = json.dumps({"data": img.tolist()})

    resp = requests.post(service.scoring_uri, input_data, headers=headers)

    result = json.loads(resp.text)['result']
    print("Actual = {}".format(label))
    print(result)
    result = np.argmax(result)
    print(result)

Actual = 0
[[0.001830501016229391, 9.361144748254446e-07, 0.9981685876846313, 2.0869115168739094e-15, 4.370818377369412e-12]]
2
Actual = 0
[[0.17992272973060608, 0.18711715936660767, 0.6285001039505005, 2.3066599741383698e-09, 0.004460045136511326]]
2
Actual = 1
[[0.00026079663075506687, 1.5959099982865155e-05, 0.999715268611908, 6.733519342105865e-08, 7.894039299571887e-06]]
2
Actual = 1
[[0.049205657094717026, 0.336171954870224, 0.6133489608764648, 0.0012546161888167262, 1.884485209302511e-05]]
2
Actual = 2
[[6.398854748113081e-05, 0.9871670603752136, 0.012170684523880482, 5.552548714149452e-07, 0.0005977004184387624]]
1
Actual = 2
[[7.139770787034649e-06, 2.941611683127121e-07, 0.9994534850120544, 0.0005291865090839565, 9.878116543404758e-06]]
2
Actual = 3
[[0.9905893802642822, 0.006941755767911673, 0.0023646678309887648, 0.00010410315007902682, 9.285516000545613e-08]]
0
Actual = 3
[[0.9905893802642822, 0.006941755767911673, 0.0023646678309887648, 0.00010410315007902682, 9.285516000

In [115]:
# 7898_left.jpeg - 0
from PIL import Image
from numpy import asarray

headers = {'Content-Type':'application/json'}

path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/diabetic-retinopathy/code/Users/joannejons/dr-color-train/'

for i in range(5):
    img = Image.open(path + test['Image'].tolist()[i])
    img = asarray(image, dtype='float32')
    img = img / 255
    img = img.reshape(-1, 512, 512, 3)

    input_data = json.dumps({"data": img.tolist()})


    resp = requests.post(service.scoring_uri, input_data, headers=headers)

    result = json.loads(resp.text)['result']
    print(result)
    result = np.argmax(result)
    print("Actual = {}, Prediction = {}".format(test['Labels'].tolist()[i], result))



# img = Image.open(path)
# img = asarray(image, dtype='float32')
# img = img / 255
# img = img.reshape(-1, 512, 512, 3)

# print(img.shape)




input_data = json.dumps({"data": img.tolist()})

# headers = {'Content-Type':'application/json'}

# resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)

# print(resp.text)
# result = json.loads(resp.text)['result']
# result = np.argmax(result)
# print(result)



[[0.4736922085285187, 0.5263077616691589]]
Actual = 0, Prediction = 1
[[0.4736922085285187, 0.5263077616691589]]
Actual = 1, Prediction = 1
[[0.4736922085285187, 0.5263077616691589]]
Actual = 0, Prediction = 1
[[0.4736922085285187, 0.5263077616691589]]
Actual = 0, Prediction = 1
[[0.4736922085285187, 0.5263077616691589]]
Actual = 0, Prediction = 1
POST to url http://99e36a78-8ca7-49ea-8029-0819c78f80d9.eastus2.azurecontainer.io/score


In [112]:
test.shape

(3726, 2)

In [ ]:
from PIL import Image
from numpy import asarray

img = Image.open(path)
img = asarray(image, dtype='float32')
img = img / 255
img = img.reshape(-1, 512, 512, 3)

headers = {'Content-Type':'application/json'}
input_data = json.dumps({"data": img.tolist()})

resp = requests.post(service.scoring_uri, input_data, headers=headers)

result = json.loads(resp.text)['result']
result = np.argmax(result)